# N-Layer Neural Network | Text Colour Predictor

Task:
- Build a scalable feed-forward neural network.
- Input values of RGB 'background colour'.
- Predict if light or dark coloured text should be used over the RGB colour to make the text readable.

Task mapping:
- Objects of interest: RGB vectors (3 $\times$ 1 dimension).
- Labels: '1' light text versus '0' dark text.

Resources:
- [Feed-forward NN playground](https://playground.tensorflow.org)
- [7 types of activation functions](https://missinglink.ai/guides/neural-network-concepts/7-types-neural-network-activation-functions-right/)
- [Activation Functions in Neural Networks](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6)
- [Comprehensive list of activation functions...](https://stats.stackexchange.com/questions/115258/comprehensive-list-of-activation-functions-in-neural-networks-with-pros-cons)
- [A Practical Guide to ReLU](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7)
- [What should I do when my neural network doesn't learn?](https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn)

## Imports

In [ ]:
# Libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Scripts
from rgb import *
from n_layer import *

## Data visualisation

In [ ]:
# Test the RGB class and data visualisation tool functions:
colours = generate_rgb_data(size=1, extreme=True, extreme_magnitude=200)

for colour in colours:
    colour.generate_img(font_col='#fff')
    plt.imshow(colour.img)
    print('RGB:', colour.RGB, 'Hex:', colour.hex)

## Generate data

NB: Change "extreme_magnitude" to adjust the level of noise. Default has no noise (i.e. easy to model).

In [ ]:
np.random.seed(42) # Optional: set seed for data generation.
extreme_magnitude = 100 # Optional: set magnitude lower for higher error.

colours = generate_rgb_data(size=500, extreme=True, extreme_magnitude=extreme_magnitude)
data = pd.DataFrame([x.RGB for x in colours], columns=['R', 'G', 'B'])

display('Training set:', data)

## Assign Labels with 'lazy' method

In [ ]:
clusterer = AgglomerativeClustering(n_clusters=2, linkage='ward').fit(data.values)
y = clusterer.labels_

In [ ]:
for i, label in enumerate(y[499:]):
    if label == 1:  # NB: must check for most appriate label-to-class assignment.
        print('---> light text')
        colours[i].generate_img(font_col='#fff')
    else:
        print('---> dark text')
        colours[i].generate_img(font_col='#000')
    plt.imshow(colours[i].img)
    plt.show()

## Train/test split

In [ ]:
# Split data into training & testing sets:
train_temp, test_temp = train_test_split(data.join(pd.Series(y, name='y')))

display(train_temp.head(5))
display(test_temp.head(5))

## Preprocessing

In [ ]:
train = pd.DataFrame(np.insert(normalize(X=train_temp.values[:, :3]), 3, train_temp.values[:, 3], axis=1),
                     index=train_temp.index,
                     columns=train_temp.columns)

test = pd.DataFrame(np.insert(normalize(X=test_temp.values[:, :3]), 3, test_temp.values[:, 3], axis=1),
                    index=test_temp.index,
                    columns=test_temp.columns)

display(train.head(5))
display(test.head(5))

## Use NN on a dummy example

NB: the idea behind using the dummy example is that it is easy to calculate by hand.

The results should be as follows:
- First ***forward pass*** output: $\begin{bmatrix} 2 \\ 2 \end{bmatrix}$
- ***Backpropagation***...
    - ... hidden layer update: $\begin{bmatrix} -1 & 0.1 \\ 0 & 0.8 \end{bmatrix}$
    - ... output layer update: $\begin{bmatrix} 0.9 & -0.2 \\ -1.2 & 0.6 \end{bmatrix}$
- Second ***forward pass*** output: $\begin{bmatrix} 1.66 \\ 0.32 \end{bmatrix}$

In [ ]:
# Setup inputs:
X = np.array([0, 1]).reshape((2,1))
y = np.array([1, 0]).reshape((2,1))
Ws = [np.array([[-1, 0], [0, 1]], dtype=float),
      np.array([[1, 0], [-1, 1]], dtype=float)]

# Initialise NN:
NN = NeuralNetwork(x_input=X,
                   y_input=y,
                   bias=1,
                   eta=0.1,
                   n_nodes=2,
                   n_layers=2,
                   weights=Ws,
                   linear=True)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

NN.backprop()
print('\nBackpropagation:\nhidden:\n{}\noutput:\n{}'.format(NN.weights[-2], NN.weights[-1]))

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

## Train NN on the training set

In [ ]:
# Setup inputs:
input_X = train.values[0, :3].reshape((3,1))
input_y = train.values[0, 3].reshape((1,1))
print('X:\n{}\ny:\n{}'.format(input_X, input_y))

# Initialise NN:
NN = NeuralNetwork(x_input=input_X,
                   y_input=input_y,
                   bias=1,
                   eta=0.1,
                   n_nodes=5,
                   n_layers=3,
                   weights=None,
                   linear=False)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

NN.backprop()
print('\n...')

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

In [ ]:
# Setup inputs:
tr_i = train.shape[0]
train_Xs = train.values[:, :3].reshape((tr_i, 3, 1))
train_ys = train.values[:,  3].reshape((tr_i, 1, 1))
print('Shapes of inputs:', train_Xs.shape, train_ys.shape)

# Train NN:
train_y_preds, train_y_errors = NN.fit(x_inputs=train_Xs, y_inputs=train_ys, iterations=1000)

In [ ]:
# Check training predictions:
print('Shape of y_preds:', train_y_preds.shape)
print(train_y_preds.reshape(tr_i)[:5])

# Check traininng errors:
print('Shape of y_errors:', train_y_errors.shape)
print(train_y_errors.reshape(tr_i)[:5])

In [ ]:
# Check NN layers:
for i, layer in enumerate(NN.layers):
    print('Layer #{}.\n{}\n'.format(i, layer))

In [ ]:
# Training results:
# NB: the y predictions are rounded!
train_results = pd.DataFrame({'y_true': train.y.values,
                              'y_pred': np.round(train_y_preds).reshape((tr_i,)).astype(int),
                              'same': train.y.values == np.round(train_y_preds).reshape((tr_i,)).astype(int)})

display(train_results.loc[train_results.y_pred==1].head(5))
display(train_results.loc[train_results.y_pred==0].head(5))

print('{}% error'.format(round(len(train_results[train_results.same==False]) / len(train_results) * 100)))

## Test NN on the testing set

In [ ]:
# Setup inputs:
te_i = test.shape[0]
test_Xs = test.values[:, :3].reshape((te_i, 3, 1))
test_ys = test.values[:,  3].reshape((te_i, 1, 1))
print('Shapes of inputs:', test_Xs.shape, test_ys.shape)

# Test NN:
test_y_preds = NN.predict(x_inputs=test_Xs)

In [ ]:
# Check testing predictions:
print('Shape of y_preds:', test_y_preds.shape)
print(test_y_preds.reshape(test_y_preds.shape[0])[:5])

In [ ]:
# Testing results:
# NB: the y predictions are rounded!
test_results = pd.DataFrame({'y_true': test.y.values,
                             'y_pred': np.round(test_y_preds).reshape((te_i,)).astype(int),
                             'same': test.y.values == np.round(test_y_preds).reshape((te_i,)).astype(int)})

display(test_results.head(5))

print('{}% error'.format(round(len(test_results[test_results.same==False]) / len(test_results) * 100)))

# Compare against NN via scikit-learn

In [ ]:
# Setup the sklearn neural network
sklearn_NN = MLPClassifier(activation='logistic', solver='sgd', max_iter=1000)  # NB: 'logistic' and stochastic gd chosen for fair comparison.

# Optimise parameters via a gridsearch:
sklearn_params = {'hidden_layer_sizes': [5, 50, 100], 'learning_rate_init': [0.1, 0.01, 0.001], 'alpha': [0.1, 0.01, 0.001]}
sklearn_grid = GridSearchCV(estimator=sklearn_NN, param_grid=sklearn_params, cv=5)

# Train sklearn NN:
sklearn_grid.fit(X=train.iloc[:, :3].values, y=train.y.values)
print('best parameters:\t{}'.format(sklearn_grid.best_params_))

# Test sklearn NN:
sklearn_y_preds = sklearn_grid.predict(X=test.iloc[:, :3].values)

# Display the sklearn predictions:
sklearn_results = pd.DataFrame({'y_true': test.y.values, 'y_pred': sklearn_y_preds, 'same': test.y.values == sklearn_y_preds})
display(sklearn_results.head(5))

# Print error:
print('{}% error'.format(round(len(sklearn_results[sklearn_results.same==False]) / len(sklearn_results) * 100)))